In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm#进度条工具

import warnings
warnings.filterwarnings("ignore")

import re

In [27]:
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def check_link(url):  
    try:            
        r = requests.get(url)  
        r.raise_for_status()  #抛出异常
        r.encoding = r.apparent_encoding  #解决编码问题
        return r.text  #读取文本多有r.text 读取图片多用r.content二进制
    except:  
        print('无法链接服务器！！！')  

In [3]:
def get_index(demo_df,name):
    return [ (indexs,i)  for indexs in demo_df.index for  i in range(len(demo_df.loc[indexs].values)) if(demo_df.loc[indexs].values[i] ==name)]  

In [4]:
def get_urls_from_pageURL(pageURL):
    rs = check_link(pageURL)
    soup = BeautifulSoup(rs,'html.parser')
    table = soup.find(id='table5')
    urls = {re.search(r'\d.*日',i.contents[0].contents[0]).group(): pageURL+i.attrs['href'] for i in table.find_all('a')}
    #词典名为链接文本中的日期'\d.*日'匹配数字为开头以日为结束的日期如2018年6月1日至2018年6月15日
    #标签'a'为html中的链接'href'为链接值
    return urls

In [5]:
url_2018 = 'http://www.nifdc.org.cn/directory/web/WS02/CL0108/'
url_2017 = 'http://www.nifdc.org.cn/directory/web/WS02/CL0873/'
url_2016 = 'http://www.nifdc.org.cn/directory/web/WS02/CL0833/'

In [6]:
urls = {}
for url in [url_2018,url_2017,url_2016]:
    urls.update(get_urls_from_pageURL(url))
#urls中为2018、2017、2016三年所有的审批链接

In [7]:
%%time
df = pd.DataFrame()
for i,u in tqdm(enumerate(urls.keys())):
    
    d = pd.read_html(urls[u],header=0)[16]#经测试，网页中表格在第16个位置
    d.rename(columns={'制品名称':'产品名称','序 号':'序号','批签发证号':'证书编号','批量/进口量':'签发量','报告书编号':'报告编号'},inplace=True)
    #第一遍爬取后发现表格中的列名存在异名，在此替换成同名
    if i > 0 :
        #为了判断本次读取到的表格的列名是否是上次多数相同，不同年份的表格列数不同，故测试选择了大小为5
        x = set(d.columns.values)
        y = set(df.columns.values)
        if len(x.intersection(y)) < 5:
            #列名不匹配有两种情况
            location = get_index(d,'制品名称')
            if location:
                #html表格元素有问题，列名不在第一行，在后面的某行
                d = d.iloc[location[0][0]:,location[0][1]:].reset_index(drop=True).T.set_index(0).T.reset_index(drop=True)
                d.rename(columns={'制品名称':'产品名称','序 号':'序号','批签发证号':'证书编号','批量/进口量':'签发量','报告书编号':'报告编号'},inplace=True)
            else:
                #根本不存在该表格
                print(u,d.shape,'error')
                continue    
    df = pd.concat([df,d])            

58it [06:44,  6.97s/it]

2016年9月16日至9月31日 (42, 10) error


59it [06:44,  6.86s/it]

2016年9月1日至9月15日 (42, 10) error


75it [08:32,  6.84s/it]

CPU times: user 5min 33s, sys: 1.33 s, total: 5min 35s
Wall time: 8min 32s


In [13]:
df.dropna(how='all',axis=1,inplace=Tru
df.drop(['序号','拒签量'],axis=1,inplace=True)

In [14]:
df.notnull().sum()

产品名称         12320
批号           12319
批签发机构         2754
批量（人份）        2745
报告编号          1457
收检编号          2754
有效期至         12320
检品编号          4606
生产企业         12320
签发日期          9072
签发结论         11938
签发结论(拟公布)      350
签发量          12318
规格           12320
证书编号          7961
dtype: int64

In [21]:
df_cs = df.loc[df.生产企业.apply(lambda x:x.find('长生')>=0)]

In [70]:
df.to_excel('20160101-20180724检验产品.xlsx',index=False,encoding='gbk')
df_cs.to_excel('长春长生公司产品.xlsx',index=False,encoding='gbk')

In [76]:
vals = df_cs.产品名称.value_counts()
vals

冻干人用狂犬病疫苗（Vero细胞）    261
水痘减毒活疫苗              166
冻干甲型肝炎减毒活疫苗           86
吸附无细胞百白破联合疫苗          34
冻干水痘减毒活疫苗             30
流感病毒裂解疫苗              27
ACYW135群脑膜炎球菌多糖疫苗      7
Name: 产品名称, dtype: int64

In [37]:
from pyecharts import Pie

In [77]:
p = Pie('长春长生公司批准产品占比',width=1000,height=1000,title_pos = 'center',title_top='top')
p.add('pie',vals.index,vals.values,radius=[0,60],is_label_show=True,is_legend_show=True,legend_orient = 'vertical',legend_pos='right',legend_top = 'bottom')
p